# 2.1 无人机sdk封装

1. **像人一样操作无人机**

    使用大模型控制无人机，类似和飞手操作无人机一样，如何让操作更简单，是应用大模型的关键。

2. **原始API过于复杂**

    目前airsim如起飞就要调用多个api函数，主要是从系统稳定性，易用性等方面考虑设计的，但对一个大模型或者飞手而言，可能更需要的是“起飞”这一个操作就行


----

根据一些研究和实践，一般需要对无人机或者类似的无人装备的sdk进行二次封装，更方便大模型调用，常见的规则如下：


1. **接口语义化封装**
将底层坐标系转换（如AirSim的z轴负方向）封装在接口内部，对外暴露自然语义（如`fly_to([x,y,正数高度]`）），类似代码中`fly_to()`对z值的处理逻辑。

2. **功能原子化**
每个方法对应独立操作单元（如takeoff/land单独封装），避免复合操作，方便大模型按需调用（类似`set_yaw()`与`fly_to`的分离）

3. **参数标准化**
统一使用基础数据类型（list/dict）作为输入输出，如位置统一返回[x,y,z]列表，降低大模型处理复杂度（参见`get_drone_position()`返回值设计）

4. **异常静默处理**
内置重试机制和默认值返回，如`get_position()`中的while循环规避空值，避免调用链路中断

5. **物理单位统一**
内部统一转换量纲（如弧度转角度），对外始终使用直观单位（如`get_yaw()`返回角度值）

6. **上下文保持**
维持client连接状态（__init__中初始化连接），通过类属性保存会话上下文，确保多指令调用的连续性

7. **语义别名映射**
建立对象字典（objects_dict）实现自然语言到引擎对象的转换，提升提示词相容性

8. **异步转同步**
通过.join()将异步操作转为同步（如takeoffAsync().join()），避免大模型处理异步回调的复杂性

9. **传感器抽象**
对原始传感器数据（如Lidar点云）进行预处理，返回可直接使用的指标数据（`get_distance()`返回最小距离值）

10. **安全边界设置**
在关键操作（如reset）中保持API控制权，通过`enableApiControl`等机制防止失控

建议补充规则：增加方法调用前的前置条件校验（如起飞前检查解锁状态）、返回结构标准化（统一包含success标志位）、增加调试模式开关等，可进一步提升大模型调用可靠性。

一些典型的封装方式如下：

In [ ]:
    def takeoff(self):
        """
        takeoff the drone
        """
        self.mav.SendPosNED(0, 0, -3, 0) 

    def land(self):
        """
        land the drone
        """
        self.mav.SendVelNED(0, 0, 0.2, 0) 
        time.sleep(1)


    def get_drone_position(self):
        """
        get the current position of the drone
        :return: position, the current position of the drone
        """
        pose = self.mav.uavPosNED
        return pose

具体可以参考rflysim_wrapper.py代码

后面如果使用专门的框架，则对函数封装形式会有更明确的要求，如：

In [ ]:
@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def get_position(object_name:str)->[float, float, float]:
    """
    get the position of a specific object
    :param object_name: the name of the object
    :return: position, the position of the object
    """
    query_string = objects_dict[object_name] + ".*"
    object_names_ue = []
    while len(object_names_ue) == 0:
        object_names_ue = self.client.simListSceneObjects(query_string)
    pose = self.client.simGetObjectPose(object_names_ue[0])
    return [pose.position.x_val, pose.position.y_val, pose.position.z_val]


## SDK代码测试，打开rflysim模拟器

In [1]:
#打开模拟器
import rflysim_wrapper
aw = rflysim_wrapper.RflySimWrapper()

No Redis labs


In [2]:
aw.takeoff()

In [3]:
aw.fly_to([20, 0, -10])
print("done")

done


In [4]:
aw.land()